In [26]:
import pandas as pd

In [27]:
df1 = pd.read_excel("all_mpnet_base_v2结果.xlsx")

In [28]:
import re
# 合并df1、df2和df3的值到一个列表中
mylist1 = list(df1.values.flatten())
# 使用正则表达式提取所有单引号内的内容
pattern = re.compile(r"'(.*?)'")
content_set = set()
for item in mylist1:
    content_set.update(pattern.findall(item))
# 将集合转换为列表
content_list = list(content_set)
# 打印结果
len(content_list)

87

In [29]:
content = pd.DataFrame(content_list, columns=['words'])

In [30]:
content[df1.columns] = None

In [31]:
# 遍历每一列
for j, column in enumerate(df1.columns):
    # 获取当前列的数据
    column_data = df1[column]
    # 遍历每一个元素
    for i, value in enumerate(column_data):
        pattern = re.compile(r"'(.*?)'")
        element_list = pattern.findall(value)
        element = ''.join(element_list)
        pattern2 = re.compile(r', (.*?)\)')
        element_list2 = pattern2.findall(value)
        element2 = float(element_list2[0])
        # 检查元素是否存在于 content 中
        if element in content['words'].tolist():
            # 如果存在，则将值追加到对应的列
            content.loc[element, df1.columns[j]] = element2

In [32]:
# content.to_excel("content.xlsx")

In [33]:
df2 = content.drop(content.index[0:len(content_list)])
df2 = df2.drop(columns=['words'])
df2

,S0,S1,S2,S3,S4,S5,M0,M1,M2,M3,M4,M5,L0,L1,L2,L3,L4,L5
polymer,0.569,0.5843,0.5965,0.3964,0.5016,0.4226,0.5282,0.5976,0.4365,NaN,0.5035,0.6296,NaN,NaN,0.4808,0.5611,0.3843,0.612
polymers,0.566,0.5881,NaN,0.3972,NaN,NaN,NaN,0.5842,0.4411,NaN,0.5019,0.6259,NaN,0.5048,NaN,0.5725,0.3976,NaN
copolymer,0.5246,0.5655,NaN,NaN,NaN,NaN,0.4782,0.6025,0.421,NaN,NaN,0.5855,NaN,NaN,NaN,NaN,NaN,NaN
polymeric,0.5069,0.5558,0.5777,0.3639,NaN,NaN,0.5157,0.523,0.4087,NaN,0.4664,0.5898,NaN,NaN,0.5069,NaN,NaN,NaN
polyester,0.479,0.5247,0.5616,NaN,NaN,0.4264,0.5498,0.5106,NaN,0.4762,0.5237,NaN,NaN,0.5504,0.5489,0.5052,NaN,0.533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
extruders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3544,NaN
biopolymers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3527,NaN
engineered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3499,NaN
polyvinyl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4678


In [34]:
from scipy.spatial import distance
import pandas as pd

# 将缺失值设置为0
df2_filled = df2.fillna(0)

# 计算两两之间的余弦相似度
similarity_matrix = pd.DataFrame(columns=df2_filled.columns, index=df2_filled.columns)
for col1 in df2_filled.columns:
    for col2 in df2_filled.columns:
        if col1 != col2:
            vec1 = df2_filled[col1].tolist()
            vec2 = df2_filled[col2].tolist()
            similarity = 1 - distance.cosine(vec1, vec2)
            similarity_matrix.at[col1, col2] = similarity

C:\Users\RIC_ZX\AppData\Local\Temp\ipykernel_46456\18459170.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2_filled = df2.fillna(0)


In [36]:
similarity_matrix.to_excel("similarity_matrix.xlsx")

In [25]:
similarities_1_2 = pd.read_excel("S_M.xlsx", header=None)
similarities_2_3 = pd.read_excel("M_L.xlsx", header=None)

In [26]:
similarities_1_2

,0,1,2,3,4,5
0,0.443941,0.552056,0.516651,0.226417,0.441299,0.454620
1,0.599505,0.605480,0.498358,0.235697,0.398395,0.408828
2,0.448617,0.424094,0.308239,0.296406,0.438858,0.319723
3,0.193919,0.287948,0.276298,0.000000,0.280875,0.290966
4,0.129817,0.131348,0.258672,0.154205,0.217295,0.129325
5,0.234849,0.215811,0.216600,0.297706,0.226305,0.114112


# 设置阈值

In [78]:
import numpy as np
def find_similar_elements(matrix):
    matrix = matrix.to_numpy()
    rows, cols = np.where(matrix > 0.1)#设置阈值
    values = matrix[rows, cols]
    similar_matrix = np.column_stack((rows, cols, values))
    return similar_matrix

In [79]:
similar_elements_1_2 = find_similar_elements(similarities_1_2)

In [80]:
import pandas as pd
# 创建新的 DataFrame
similar_elements_1_2_pd = pd.DataFrame(similar_elements_1_2, columns=['Source', 'Target', 'Value'])
similar_elements_1_2_pd['Source'] = similar_elements_1_2_pd['Source'].apply(lambda x: 'S' + str(int(x)))
similar_elements_1_2_pd['Target'] = similar_elements_1_2_pd['Target'].apply(lambda x: 'M' + str(int(x)))

In [81]:
similar_elements_1_2_pd

,Source,Target,Value
0,S0,M0,0.443941
1,S0,M1,0.552056
2,S0,M2,0.516651
3,S0,M3,0.226417
4,S0,M4,0.441299
5,S0,M5,0.454620
6,S1,M0,0.599505
7,S1,M1,0.605480
8,S1,M2,0.498358
9,S1,M3,0.235697


In [82]:
similar_elements_2_3 = find_similar_elements(similarities_2_3)

In [83]:
import pandas as pd
# 创建新的 DataFrame
similar_elements_2_3_pd = pd.DataFrame(similar_elements_2_3, columns=['Source', 'Target', 'Value'])
similar_elements_2_3_pd['Source'] = similar_elements_2_3_pd['Source'].apply(lambda x: 'M' + str(int(x)))
similar_elements_2_3_pd['Target'] = similar_elements_2_3_pd['Target'].apply(lambda x: 'L' + str(int(x)))

In [84]:
# 桑基图矩阵
similar_elements_1_2_3_pd = pd.concat([similar_elements_1_2_pd, similar_elements_2_3_pd])

In [85]:
similar_elements_1_2_3_pd

,Source,Target,Value
0,S0,M0,0.443941
1,S0,M1,0.552056
2,S0,M2,0.516651
3,S0,M3,0.226417
4,S0,M4,0.441299
...,...,...,...
25,M5,L1,0.192801
26,M5,L2,0.197130
27,M5,L3,0.233165
28,M5,L4,0.210301


In [86]:
import pandas as pd

# 合并所有节点
nodes = pd.Series(similar_elements_1_2_3_pd['Source'].tolist() + similar_elements_1_2_3_pd['Target'].tolist()).unique()

# 为节点分配唯一索引
node_index = {node: index for index, node in enumerate(nodes)}

# 替换 Source 和 Target 列
similar_elements_1_2_3_pd['source'] = similar_elements_1_2_3_pd['Source'].apply(lambda x: node_index[x])
similar_elements_1_2_3_pd['target'] = similar_elements_1_2_3_pd['Target'].apply(lambda x: node_index[x])

# 转换数据格式
links = similar_elements_1_2_3_pd[['source', 'target', 'Value']]

In [36]:
from pyecharts.charts import Sankey
from pyecharts import options as opts

# 从 similar_elements_2_3_pd 中读取数据
data = similar_elements_1_2_3_pd.values.tolist()

nodes = []
links = []

for item in data:
    source = item[0]
    target = item[1]
    value = item[2]

    if source not in nodes:
        nodes.append(source)
    if target not in nodes:
        nodes.append(target)

    links.append({"source": source, "target": target, "value": value})

sankey = (
    Sankey()
    .add(
        series_name="",
        nodes=[{"name": node} for node in nodes],
        links=links,
        linestyle_opt=opts.LineStyleOpts(opacity=0.5, curve=0.3, color="source"),
        #itemstyle_opts=opts.ItemStyleOpts(border_width=1, border_color="#aaa")
    )
    .set_global_opts(title_opts=opts.TitleOpts(title=""),
                     #theme="wonderland" 
                     )
     
)

sankey.render_notebook()

'C:\\Users\\RIC_ZX\\Desktop\\papers\\ZLW\\SBERT\\TR三维KMeans\\SBERT-TextRank\\sankey_chart.html'